In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD

from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

In [2]:
df_1=pd.read_parquet("../Datasets/df_1.parquet")

In [12]:
df_1.sample(11)

,id,user,score
10963661,as2247,123800,3.5
6418756,ds1370,66258,4.0
1227881,as6369,12611,2.0
10521400,hs1071,119323,1.0
2955449,as2297,30799,2.5
379849,ns8475,4017,5.0
3560557,as9137,37116,4.0
5249565,as1266,54086,3.0
7202492,as3798,74275,3.5
1799554,as8983,18652,3.0


In [5]:
df_1=df_1[["id","user","score"]]

In [7]:
# Se modela df con el total de los valores de score

read = Reader(rating_scale=(0.5,5))

data = Dataset.load_from_df(df_1, reader=read)

In [8]:
# se separa los datos

trainset, testset = train_test_split(data, test_size=0.25)

In [9]:
# Se usa modelo de Singular Value Decomposition

svd = SVD()

In [10]:
# Se entrena el modelo

svd.fit(trainset)

In [11]:
# Se realiza una predicción con los datos testeados

predictions = svd.test(testset)

In [13]:
# Hago la predicción con un userId, una id al azar y un score al azar
user_id = '66258'
movie_id = 'ds1370'
r_ui = 4
svd.predict(user_id, movie_id, r_ui=r_ui, verbose=True)

user: 66258      item: ds1370     r_ui = 4.00   est = 3.53   {'was_impossible': False}


Prediction(uid='66258', iid='ds1370', r_ui=4, est=3.533636823010311, details={'was_impossible': False})

In [14]:
# Se realiza una validación cruzada

cross_validate(svd, data, measures=['RMSE', 'MAE'], cv = 3, verbose=True)